# IIJA All Projects 

In [1]:
# ! pip install nltk
# ! pip install textblob

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from shared_utils import geography_utils
from dla_utils import _dla_utils

from calitp import to_snakecase

import utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)


## Read Data

In [4]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [5]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/CopyofFMIS_Projects_Universe_IIJA_Reporting_4.xls", 
                           # sheet_name='FMIS 5 Projects  ', header=[3]
                           sheet_name='IIJA',
                           # sheet_name='FMIS 5 Projects  ',
                           ))

In [6]:
proj.drop(columns =['unnamed:_0', 'unnamed:_13', 'unnamed:_14'], axis=1, inplace=True)

In [7]:
proj.sample(5)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value
147,2022-09-07,Y003,PROJ TO REDUCE PM 2.5 EMI IIJA,5085055,1021000034L,"NOTTINGHAM LANE (FROM OLIVE TO HANSON), TAMWORTH CT, BUCKINGHAM CT, AND ALEXANDER AVE. INSTALL FIVE-FOOT SIDEWALK ADJACENT TO EXISTING CURB AND GUTTE",47.00,Cong Dist 16,28.00,Facilities for Pedestrians and Bicycles,"470,813.00",L5085MCAG
1037,2022-08-30,Y237,"STBG 5K-49,999 POP IIJA",X093010,0217000009S,IN SISKIYOU COUNTY IN YREKA ON ROUTE 3 FROM 0.4 MILE NORTH OF LAURA LANE TO JUNIPER DRIVE AND ON ROUTE 263 FROM ROUTE 3 TO 1.0 MILE SOUTH OF LONG GUL,93.00,Cong Dist 1,6.00,4R - Restoration & Rehabilitation,"24,311,937.39",S NON-MPO
412,2022-07-25,Y240,SURFAC TRNSP BLK GRTS-FLX IIJA,5933143,0418000118L,"IN CASTRO VALLEY: ON ANITA AVENUE BETWEEN CASTRO VALLEY BLVD. AND SOMERSET AVENUE CONSTRUCT SIDE WALKS,CURBS, GUTTERS, DRIVEWAYS, PEDESTRIAN RAMPS AN",1.00,Cong Dist 15,17.00,Construction Engineering,"1,000.00",L5933MTC
664,2022-08-08,YS30,HIGHWAY SAFETY IMP PROG IIJA,P020179,0300020624S,"NEAR LAKE WILDWOOD FROM 0.4 MILE EAST OF MCGANNERY LANE TO YUBA/NEVADA COUNTY LINE; ALSO IN NEVADA COUNTY, FROM YUBA/NEVADA COUNTY LINE TO MOONEY FLA",115.00,Cong Dist 3,17.00,Construction Engineering,"278,000.00",S SACOG
423,2022-04-04,Y233,STBG IIJA OFF-SYSTEM BRIDGE,5937058,04923198L,ALAMITOS RD. BRIDGE @ ALAMITOS CK (37C0159) BRIDGE REPLACEMENT/SEISMIC RET (TC),85.00,Cong Dist 19,17.00,Construction Engineering,"1,075,808.00",L5937MTC


In [8]:
proj = proj.dropna(how='all') 

In [9]:
proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].fillna(value='None')

In [10]:
proj.head(1)

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value
0,2022-03-03,Y230,STBG-URBANIZED >200K IIJA,5002174,0314000222L,BELL AVE. FROM BOLLENBACHER AVE. TO ASTORIA ST ROAD REHABILITATION (TC),67.00,Cong Dist 6,4.00,4R - No Added Capacity,"2,841,397.00",L5002SACOG


In [11]:
len(proj)

1241

In [12]:
## agency identifier column doesnt always have locode

## Function Work

In [13]:
# (proj[proj['summary_recipient_defined_text_field_1_value'].str.startswith('L')]).sample()

In [14]:
# (proj[proj['summary_recipient_defined_text_field_1_value'].str.startswith('L')]).sample()

In [15]:
#seeing if the space is the way to filter
# ((proj[~proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")])>>count(_.summary_recipient_defined_text_field_1_value))

In [16]:
#check to make sure there are numbers
# ((proj[~proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")])>>count(_.summary_recipient_defined_text_field_1_value)).iloc[1:20]

In [17]:
# locode_proj = ((proj[~proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))
# locode_proj = locode_proj>>filter(_.summary_recipient_defined_text_field_1_value!='None')

In [18]:
# len(locode_proj)

In [19]:
# locode_proj = (utils.add_name_from_locode(locode_proj, 'summary_recipient_defined_text_field_1_value'))

In [20]:
# no_locode = ((proj[proj['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))

In [21]:
# no_entry = proj>>filter(_['summary_recipient_defined_text_field_1_value']=='None')
    
#     #concat no locodes and those with no entry
# no_locode = pd.concat([no_locode, no_entry])

In [22]:
# no_locode.sample()

In [23]:
# no_locode = no_locode.rename(columns = {"recipient_name":"implementing_agency", "county_description":"county_name"})

In [24]:
# len(no_locode)

In [25]:
# pd.concat([locode_proj, no_locode])

## Function for adding Implemeting Agency Information

In [26]:
# function moved to utils

In [27]:
df = utils.identify_agency(proj, 'summary_recipient_defined_text_field_1_value')

In [28]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
216,2022-09-07,Y400,CONGESTION MITIGATION IIJA,5197043,0623000036L,INTERSECTION OF BETHEL AVE AND ALMOND AVE INSTALL NEW THREE PHASE TRAFFIC SIGNAL SYSTEM,19.00,Cong Dist 21,15.00,Preliminary Engineering,"38,953.00",L5197FCOG,"5,197.00",Sanger,6.00,Fresno County,Council of Fresno County Governments,Council Of Fresno County Goverments


In [29]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1254 entries, 0 to 671
Data columns (total 18 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   fmis_transaction_date                         1254 non-null   datetime64[ns]
 1   program_code                                  1254 non-null   object        
 2   program_code_description                      1254 non-null   object        
 3   project_number                                1254 non-null   object        
 4   recipient_project_number                      1253 non-null   object        
 5   project_title                                 1254 non-null   object        
 6   county_code                                   1254 non-null   float64       
 7   congressional_district                        1254 non-null   object        
 8   improvement_type                              1254 non-null   float64

In [30]:
df = (utils.add_description(df, 'project_title'))

In [31]:
df>>filter(_.project_type=="Project")

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_method,project_type
2,2022-08-26,Y400,CONGESTION MITIGATION IIJA,5002178,0315000001L,"SOUTH SACRAMENTO PARKWAY TRAIL- WEST. IN THE CITY OF SACRAMENTO, ADJACENT TO NORTHBOUND INTERSTATE 5, SOUTH OF POCKET ROAD/MEADOWVIEW ROAD AND FREEPO",67.00,Cong Dist 6,17.00,Construction Engineering,"50,000.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project
3,2022-08-26,Y400,CONGESTION MITIGATION IIJA,5002178,0315000001L,"SOUTH SACRAMENTO PARKWAY TRAIL- WEST. IN THE CITY OF SACRAMENTO, ADJACENT TO NORTHBOUND INTERSTATE 5, SOUTH OF POCKET ROAD/MEADOWVIEW ROAD AND FREEPO",67.00,Cong Dist 6,28.00,Facilities for Pedestrians and Bicycles,"508,614.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project
4,2022-07-15,Y400,CONGESTION MITIGATION IIJA,5002189,0316000158L,ALONG UNUSED RAIL CORRIDOR NEAR FREEPORT BLVD FROM SUTTERVILLE ROAD TO SOUTH OF MEADOWVIEW/POCKET ROAD (4.5 MILES) RAILS TO TRAILS PROJECT ; NEW CLAS,67.00,Cong Dist 6,28.00,Facilities for Pedestrians and Bicycles,"4,359,000.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,New,Project
5,2022-02-08,YS30,HIGHWAY SAFETY IMP PROG IIJA,5002199,0317000138L,"INTERSECTIONS OF 65TH ST EXPWY AT 21ST AVE, 65TH ST EXPWY AT FRUITRIDGE RD, ARDEN WAY AT HERITAGE LN, ARDEN WAY AT CHALLENGE WAY, HOWE AVE. AT SWARTH",67.00,Cong Dist 3,21.00,Safety,"1,241,500.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project
6,2022-02-08,YS30,HIGHWAY SAFETY IMP PROG IIJA,5002199,0317000138L,"INTERSECTIONS OF 65TH ST EXPWY AT 21ST AVE, 65TH ST EXPWY AT FRUITRIDGE RD, ARDEN WAY AT HERITAGE LN, ARDEN WAY AT CHALLENGE WAY, HOWE AVE. AT SWARTH",67.00,Cong Dist 6,17.00,Construction Engineering,"200,000.00",L5002SACOG,"5,002.00",Sacramento,3.00,Sacramento County,Sacramento Area Council of Governments,Sacramento Area Council Of Governments,,Project
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,2022-06-16,Y001,NATIONAL HIGHWAY PERF IIJA,4053001,0716000031S,IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE,37.00,Cong Dist 29,44.00,Other,"2,475,335.29",S SCAG,NaN,California,7.00,Los Angeles County,NaN,NaN,,Project
663,2022-06-16,Y001,NATIONAL HIGHWAY PERF IIJA,4053001,0716000031S,IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE,37.00,Cong Dist 30,17.00,Construction Engineering,"185,535.45",S SCAG,NaN,California,7.00,Los Angeles County,NaN,NaN,,Project
664,2022-06-16,Y001,NATIONAL HIGHWAY PERF IIJA,4053001,0716000031S,IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE,37.00,Cong Dist 30,42.00,Training,353.05,S SCAG,NaN,California,7.00,Los Angeles County,NaN,NaN,,Project
665,2022-06-16,Y001,NATIONAL HIGHWAY PERF IIJA,4053001,0716000031S,IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE,37.00,Cong Dist 30,44.00,Other,"448,864.71",S SCAG,NaN,California,7.00,Los Angeles County,NaN,NaN,,Project


In [32]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_method,project_type
272,2022-05-05,Y001,NATIONAL HIGHWAY PERF IIJA,P125020,1112000143S,IN SAN DIEGO COUNTY IN SAN DIEGO AND LEMON GROVE FROM 0.1 MILE SOUTH OF ELKELTON PLACE UNDERCROSSING TO ROUTE 94/125 SEPARATION. CONSTRUCT MAINTENANC,73.00,Cong Dist 53,17.00,Construction Engineering,"1,234,100.00",S SANDAG,NaN,California,11.00,San Diego County,NaN,NaN,Construct,Project


In [33]:
df.loc[df.county_name == "Statewide County", 'county_name'] = "Statewide"

## New Title Function

In [34]:
def add_new_title(df, first_col_method, second_col_type, third_col_name, alt_col_name):
    """
    Function to add new title. 
    Expected output example: "New Bike Lane in Eureka"
    """
    def return_name(df):
        
        if (df[third_col_name] == "California") & (df[alt_col_name] == "Statewide"):
            return (df[first_col_method] + " " + df[second_col_type] +" " + df[alt_col_name])
        
        elif (df[third_col_name] == "California"):
            return (df[first_col_method] + " " + df[second_col_type] + " in " + df[alt_col_name])
        
        elif (df[third_col_name] != "California"):
            return (df[first_col_method] + " " + df[second_col_type] + " in " + df[third_col_name])
        
        # elif (df[third_col_name] == "Metropolitan Transportation Commission"):
        #     return (df[first_col_method] + " " + df[second_col_type] + " in The " + df[third_col_name])

        return df

    df['project_name_new'] = df.apply(return_name, axis = 1)
    
    return df

In [35]:
df = add_new_title(df, "project_method", "project_type", "implementing_agency", "county_name")

In [36]:
df.tail()


,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_method,project_type,project_name_new
667,2022-09-09,YS30,HIGHWAY SAFETY IMP PROG IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,21.00,Safety,"4,165,800.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install,Lighting,Install Lighting in Alameda County
668,2022-09-09,YS30,HIGHWAY SAFETY IMP PROG IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,22.00,Rail/Hwy Crossing,"18,900.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install,Lighting,Install Lighting in Alameda County
669,2022-09-09,YS30,HIGHWAY SAFETY IMP PROG IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,42.00,Training,"3,000.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install,Lighting,Install Lighting in Alameda County
670,2022-09-09,YS32,SEC 164 PENALTIES HSIP IIJA,5801081,0416000125S,ALAMEDA COUNTY NEAR LIVERMORE FROM GRANT LINE ROAD UNDERCROSSING TO 0.1 MILE EAST OF NORTH FLYNN ROAD INSTALL LIGHTING SYSTEM,1.00,Cong Dist 15,17.00,Construction Engineering,"288,000.00",S MTC,NaN,California,4.00,Alameda County,NaN,NaN,Install,Lighting,Install Lighting in Alameda County
671,2022-09-23,YR10,STATE REC TRAILS ADMIN IIJA,2023000,NaN,2023 REC TRAILS PROJECT: STATE PARKS INCURRED ADMINISTRATIVE EXPENSES,999.00,Cong Dist 0,44.00,Other,"241,760.00",None,NaN,California,NaN,Statewide,NaN,NaN,,Project,Project Statewide


In [37]:
df>>filter(_.implementing_agency==("Metropolitan Transportation Commission"))>>select(_.project_name_new)

,project_name_new
518,Bike Share Program in Metropolitan Transportation Commission
519,Project in Metropolitan Transportation Commission
520,Signage in Metropolitan Transportation Commission
521,Project in Metropolitan Transportation Commission
522,Project in Metropolitan Transportation Commission
523,Project in Metropolitan Transportation Commission
524,Regional Planning Activities in Metropolitan Transportation Commission
525,Regional Planning Activities in Metropolitan Transportation Commission
526,Regional Planning Activities in Metropolitan Transportation Commission
527,Regional Planning Activities in Metropolitan Transportation Commission


### Export progress

In [38]:
#test_df = utils.title_column_names(df)

In [39]:
#test_df.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_wip_all.csv")

## Use Alt Column

In [59]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_method,project_type,project_name_new
344,2022-06-16,Y001,NATIONAL HIGHWAY PERF IIJA,Q101395,0520000169S,SANTA BARBARA COUNTY IN AND NEAR CARPINTERIA AND AT SUMMERLAND FROM 0.9 MILE SOUTH OF SOUTH PADARO LANE UNDERCROSSING TO 0.6 MILE NORTH OF PADARO LANE,83.00,Cong Dist 24,42.00,Training,"19,200.00",S SBCAG,NaN,California,5.00,Santa Barbara County,NaN,NaN,,Project,Project in Santa Barbara County


In [60]:
# using those with no match

no_match = (df>>filter(_.project_type==('Project'))>>select(_.project_number, _.recipient_project_number, _.project_title, _.improvement_type_description, _.implementing_agency, _.county_name,
                                                            _.project_method, _.project_type, _.project_name_new))

In [62]:
no_match.improvement_type_description.value_counts()

Construction Engineering                               180
Safety                                                  74
Training                                                69
4R - Restoration & Rehabilitation                       52
Planning                                                44
Preliminary Engineering                                 38
4R - Maintenance  Resurfacing                           19
Facilities for Pedestrians and Bicycles                 18
Other                                                   16
Bridge Replacement - No Added Capacity                  14
Right of Way                                            13
4R - No Added Capacity                                  12
Bridge Rehabilitation - No Added Capacity                6
4R - Added Capacity                                      6
Mitigation of Water Pollution due to Highway Runoff      4
New  Construction Roadway                                4
Bridge Replacement - Added Capacity                     

In [43]:
no_match>>filter(_.improvement_type_description=="Preliminary Engineering")

,project_title,improvement_type_description,implementing_agency,county_name,project_method,project_type,project_name_new
135,"BARTON ROAD OVERHEAD AT UNION PACIFIC RAILROAD, 0.25 MILE WEST OF I-215, BR. NO. 54C-0379 REPLACE OVERHEAD WITH TWO-LANE ROAD ACROSS DISCONTINUED RAI",Preliminary Engineering,Colton,San Bernardino County,Replace,Project,Replace Project in Colton
149,"NORTH SIDE OF CHILDS AVE (FROM GOLDEN VALLEY HIGH SCHOOL TO COFFEE ST. (WEAVER MIDDLE SCHOOL)). CONSTRUCT A CLASS I (FULLY-SEPARATED). ""SAFE ROUTES T",Preliminary Engineering,Merced,Merced County,Construct,Project,Construct Project in Merced
158,PM00222/PSTUDY002 ON VARIOUS BRIDGE LOCATIONS THROUGHOUT THE CITY OF CORONA. SEE COMMENTS FOR THE LIST OF BRIDGES. DEVELOP A BPMP PLAN FOR THE CITY-,Preliminary Engineering,Corona,Riverside County,,Project,Project in Corona
231,VARIOUS LOCATION IN SUNNYVALE PROVIDE ADVANCE DILEMMA ZONE DETECTION.,Preliminary Engineering,Sunnyvale,Santa Clara County,,Project,Project in Sunnyvale
256,US-50 AT EMPIRE RANCH ROAD CONSTRUCT 4 LANE INTERCHANGE (TC),Preliminary Engineering,Folsom,Sacramento County,Construct,Project,Construct Project in Folsom
294,"PROJECT SPANS SACRAMENTO RIVER FROM 2ND/D STREET IN WEST SACRAMENTO, UP TO THE TOP DECK OF THE I STREET BRIDGE TO THE JIBBOOM/I STREET VIADUCT AND IN",Preliminary Engineering,West Sacramento,Yolo County,,Project,Project in West Sacramento
298,CITY OF MISSION VIEJO - THE PROJECT IS LOCATED ON THE WESTERN EDGE OF MISSION VIEJO. THE MISSION VIEJO N/S CORRIDOR TRAIL IS BOUNDED BY LOS ALISOS BL,Preliminary Engineering,Mission Viejo,Orange County,,Project,Project in Mission Viejo
303,"E/B SANTA MARGARITA PARKWAY OVER ARROYO TRABUCO (BRIDGE #55C0520R) AND ANTONIO PARKWAY OVER TIJERAS CREEK (BRIDGE #55C0605), BPMP PM00035 BRIDGE PREV",Preliminary Engineering,Rancho Santa Margarita,Orange County,,Project,Project in Rancho Santa Margarita
307,"REQUA ROAD OVER HUNTER CREEK, 0.1 MI. WEST OF SR-101 REPLACE EXISTING 2-LANE BRIDGE (TC)",Preliminary Engineering,Del Norte County,Del Norte County,Replace,Project,Replace Project in Del Norte County
383,"IN EASTERN EI DORADO COUNTY, IN THE TAHOE BASIN, IN THE CITY OF SOUTH LAKE TAHOE AT EAST END OF SOUTH AVENUE, OVER THE UPPER TRUCKEE RIVER, THEN NORT",Preliminary Engineering,El Dorado County,El Dorado County,,Project,Project in El Dorado County


In [44]:
((utils.get_list_of_words((no_match>>filter(_.improvement_type_description=="4R - Restoration & Rehabilitation")), "project_title")).value_counts()).iloc[0:20]

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


mile             40
route            29
county           29
san              21
near             19
south            18
road             16
north            14
east             13
street           13
construct        13
undercrossing    13
05               13
various          11
diego            11
02               11
locations        11
los              11
separation       11
west             10
dtype: int64

In [45]:
no_match>>filter(_.project_title.str.contains("BICYC"), _.improvement_type_description!="Facilities for Pedestrians and Bicycles")

,project_title,improvement_type_description,implementing_agency,county_name,project_method,project_type,project_name_new


In [46]:
(no_match>>filter(_.improvement_type_description=="4R - Restoration & Rehabilitation"))

,project_title,improvement_type_description,implementing_agency,county_name,project_method,project_type,project_name_new
79,OAKPORT ST FROM EDGEWATER TO I-880 FREEWAY ENTRANCE; OAKLAND AVE FROM ORANGE STREET TO MACARTHUR; MONTEREY BLVD FROM MAIDEN LN TO BENNET PL IN OAKLAN,4R - Restoration & Rehabilitation,Oakland,Alameda County,,Project,Project in Oakland
183,EAST DUNNE AVENUE BETWEEN LOWER THOMAS GRADE AND THE EASTERN CITY LIMIT. FEDERAL PARTICIPATING LOCATION SEGMENTS: 1) LOWER THOMAS GRADE TO FLAMING OA,4R - Restoration & Rehabilitation,Morgan Hill,Santa Clara County,,Project,Project in Morgan Hill
232,"IN EAST SUNNYVALE SENSE OF PLACE PLAN AREA: ON DUANE AVENUE FROM FAIR OAKS AVENUE TO LAWRENCE EXPRESSWAY, STEWART DRIVE FROM WOLFE ROAD TO DUANE AVEN",4R - Restoration & Rehabilitation,Sunnyvale,Santa Clara County,,Project,Project in Sunnyvale
247,"BELMONT: CHULA VISTA FROM ALAMEDA DE LAS PULGAS TO RALSTON AVE, 6TH AVENUE FROM RALSTON AVENUE TO HILL STREET, 6TH AVENUE FROM EMMETT AVENUE TO HARBO",4R - Restoration & Rehabilitation,Belmont,San Mateo County,,Project,Project in Belmont
281,"COLEMAN CREEK PATH @ SNYDER LANE, HINEBAUGH CREEK PATH @ COUNTRY CLUB DRIVE, HINEBAUGH CREEK PATH @ STATE FARM DRIVE, HINEBAUGH CREEK PATH @ COMMERCE",4R - Restoration & Rehabilitation,Rohnert Park,Sonoma County,,Project,Project in Rohnert Park
299,"GOLDEN VALLEY PARKWAY (TOWNE CENTRE DRIVE AND LATHROP ROAD), RIVER ISLANDS PARKWAY (GOLDEN VALLEY PARKWAY LATHROP FIRE STATION 34), LATHROP ROAD. ROA",4R - Restoration & Rehabilitation,Lathrop,San Joaquin County,,Project,Project in Lathrop
579,IN AND NEAR THE CITY OF MODESTO ON SR 132 NEAR NORTH DAKOTA AVE TO NEAR 6TH ST AND ON SR 99 FROM 0.5 M SOUTH OF TUOLUMNE RIVER BRIDGE TO 0.1 M NORTH,4R - Restoration & Rehabilitation,Modesto,Stanislaus County,,Project,Project in Modesto
1,ALAMEDA COUNTY AT SUNOL AND IN PLEASANTON AND DUBLIN PRECAST CONCRETE PAVEMENT AND HMA,4R - Restoration & Rehabilitation,California,Alameda County,Pave,Project,Pave Project in Alameda County
4,"IN SAN DIEGO COUNTY IN CHULA VISTA, FROM 0.3 MILE SOUTH OF MAIN STREET UNDERCROSSING TO PALOMAR STREET OVERCROSSING CONVERT POTABLE IRRIGATION SYSTEM",4R - Restoration & Rehabilitation,California,San Diego County,,Project,Project in San Diego County
6,"IN SAN DIEGO COUNTY IN CHULA VISTA, FROM 0.3 MILE SOUTH OF MAIN STREET UNDERCROSSING TO PALOMAR STREET OVERCROSSING CONVERT POTABLE IRRIGATION SYSTEM",4R - Restoration & Rehabilitation,California,San Diego County,,Project,Project in San Diego County


### Function for no_matches

In [154]:
def update_no_matched(df, flag_col, desc_col, name_col):
    """
    function to itreate over projects that did not match the first time
    using an existing project's short description of project type. 
    """
    
    def return_project_type(df):
        
        if (df[flag_col] == "Project") & (df[desc_col] == "Bridge Rehabilitation") | (df[desc_col] =="Bridge Rehabilitation - No Added Capacity") | (df[desc_col] =="Bridge Rehabilitation - Added Capacity"):
            return ("Bridge Rehabilitation in " + df[name_col])
        
        elif (df[flag_col] == "Project") & (df[desc_col] == "Facilities for Pedestrians and Bicycles"):
            return (df[desc_col] + " in " + df[name_col])
        
        elif (df[flag_col] == "Project") & (df[desc_col] == "Safety"):
            return (df[desc_col] + " Improvement Projects in " + df[name_col])
        
        # elif (df[flag_col] == "Project") & (df[flag_col] == "Planning") & (df[title_col].str.contains("REGION CONSOLIDATED PLANNING GRANT")):
        #     return ("Regional Planning Grant in " + df[name_col])
            
        elif (df[flag_col] == "Project") & (df[desc_col] == "Planning "):
            return "Project Planning in " + df[name_col]
            
        elif (df[flag_col] == "Project") & (df[desc_col] == "Preliminary Engineering"):
            return (df[desc_col] + " Projects in " + df[name_col])
        
        elif (df[flag_col] == "Project") & (df[desc_col] == "Construction Engineering"):
            return (df[desc_col] + " Projects in " + df[name_col])
        
        elif (df[flag_col] == "Project") & (df[desc_col] == "4R - Restoration & Rehabilitation"):
            return ("Road Restoration & Rehabilitation in " + df[name_col])
        
        elif (df[flag_col] == "Project") & (df[desc_col] == "4R - Maintenance  Resurfacing"):
            return ("Maintenance Resurfacing in " + df[name_col])
        
        elif (df[flag_col] == "Project") & (df[desc_col] == "Bridge Replacement - Added Capacity") | (df[desc_col] == "Bridge Replacement - No Added Capacity") | (df[desc_col] == "Bridge New Construction") | (df[desc_col] == "Special Bridge"):
            return ("Bridge Replacement in " + df[name_col])
        
        elif (df[flag_col] == "Project") & (df[desc_col] == "Mitigation of Water Pollution due to Highway Runoff"):
            return (df[desc_col] + " in " + df[name_col])
        
        elif (df[flag_col] == "Project") & (df[desc_col] == "4R - Added Capacity"):
            return ("Added Roadway Capacity in " + df[name_col])
        
        elif (df[flag_col] == "Project") & (df[desc_col] == "4R - No Added Capacity"):
            return ("Road Construction in " + df[name_col])
        
        elif (df[flag_col] == "Project") & (df[desc_col] == "New  Construction Roadway"):
            return ("New Construction Roadway in " + df[name_col])
        
        elif (df[flag_col] == "Project") & (df[desc_col] == "Traffic Management/Engineering - HOV"):
            return ("Traffic Management Project in " + df[name_col])
        
        # elif (df[flag_col] == "Project") & (df[desc_col] == "Training"):
        #     return (df[desc_col] + " for ___ in " + df[name_col])
        
        else:
            return "" #(df[desc_col] + " in " + df[name_col])

        return df


    df['project_name_new2'] = df.apply(return_project_type, axis = 1)
    
    #df.apply(func, axis=1)
    
    return df

In [155]:
no_match_title = (update_no_matched(no_match, 'project_type', 'improvement_type_description', 'implementing_agency'))

In [156]:
no_match_title.sample()

,project_number,recipient_project_number,project_title,improvement_type_description,implementing_agency,county_name,project_method,project_type,project_name_new,project_name_new2
655,2801153,0416000024S,"SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.",Construction Engineering,California,Santa Clara County,,Project,Project in Santa Clara County,Construction Engineering Projects in California


In [50]:
# remaining = (no_match_title>>filter(_.project_name_new2=='')).improvement_type_description.value_counts()
# remaining

### Remaining 
| Description | Count |
| --- | --- |
|Training | 69 |
| Other  |  16 |
| Right of Way | 13 |
|  Safety and Education for Peds/Bicyclist | 1 |
| Bridge Inspection and Bridge Related Training | 1 |
| Rail/Hwy Crossing    |  1 |
| Utilities    | 1 |
| Landscaping and Other Scenic Beautification   |    1 |
| Research   |  1 |


In [58]:
df>>filter(_.project_title.str.contains("CHOKEPOINT RELIEF"))

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name,project_method,project_type,project_name_new
576,2022-09-22,Y460,NATIONAL HWY FREIGHT PROG IIJA,P057073,0715000076S,IN LOS ANGELES COUNTY. ROUTE 57/60 CONFLUENCE CHOKEPOINT RELIEF PROGRAM. RECONSTRUCT GRAND AVENUE OVERCROSSING. RECONSTRUCT NORTHBOUND SR-57 CONNECTO,37.00,Cong Dist 39,11.00,Bridge Replacement - No Added Capacity,"72,969,500.00",L6065SCAG,"6,065.00",Los Angeles County Metropolitan Transportation Authority,7.00,Los Angeles County,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments,Construct,Project,Construct Project in Los Angeles County Metropolitan Transportation Authority
577,2022-09-22,Y460,NATIONAL HWY FREIGHT PROG IIJA,P057073,0715000076S,IN LOS ANGELES COUNTY. ROUTE 57/60 CONFLUENCE CHOKEPOINT RELIEF PROGRAM. RECONSTRUCT GRAND AVENUE OVERCROSSING. RECONSTRUCT NORTHBOUND SR-57 CONNECTO,37.00,Cong Dist 39,17.00,Construction Engineering,"1,000,000.00",L6065SCAG,"6,065.00",Los Angeles County Metropolitan Transportation Authority,7.00,Los Angeles County,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments,Construct,Project,Construct Project in Los Angeles County Metropolitan Transportation Authority
578,2022-09-22,Y460,NATIONAL HWY FREIGHT PROG IIJA,P057073,0715000076S,IN LOS ANGELES COUNTY. ROUTE 57/60 CONFLUENCE CHOKEPOINT RELIEF PROGRAM. RECONSTRUCT GRAND AVENUE OVERCROSSING. RECONSTRUCT NORTHBOUND SR-57 CONNECTO,37.00,Cong Dist 39,42.00,Training,"30,500.00",L6065SCAG,"6,065.00",Los Angeles County Metropolitan Transportation Authority,7.00,Los Angeles County,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments,Construct,Project,Construct Project in Los Angeles County Metropolitan Transportation Authority


In [52]:
no_match_title>>filter(_.improvement_type_description=='Training')

,project_title,improvement_type_description,implementing_agency,county_name,project_method,project_type,project_name_new,project_name_new2
578,IN LOS ANGELES COUNTY. ROUTE 57/60 CONFLUENCE CHOKEPOINT RELIEF PROGRAM. RECONSTRUCT GRAND AVENUE OVERCROSSING. RECONSTRUCT NORTHBOUND SR-57 CONNECTO,Training,Los Angeles County Metropolitan Transportation Authority,Los Angeles County,Construct,Project,Construct Project in Los Angeles County Metropolitan Transportation Authority,Training for ___ in Los Angeles County Metropolitan Transportation Authority
580,IN AND NEAR THE CITY OF MODESTO ON SR 132 NEAR NORTH DAKOTA AVE TO NEAR 6TH ST AND ON SR 99 FROM 0.5 M SOUTH OF TUOLUMNE RIVER BRIDGE TO 0.1 M NORTH,Training,Modesto,Stanislaus County,,Project,Project in Modesto,Training for ___ in Modesto
3,ALAMEDA COUNTY AT SUNOL AND IN PLEASANTON AND DUBLIN PRECAST CONCRETE PAVEMENT AND HMA,Training,California,Alameda County,Pave,Project,Pave Project in Alameda County,Training for ___ in California
11,"ALAMEDA COUNTY IN SAN LEANDRO AT WASHINGTON AVENUE OVERCROSSING ROADWAY EXCAVATION, HMA, AND MODIFYING ELECTRICAL SYSTEMS.",Training,California,Alameda County,,Project,Project in Alameda County,Training for ___ in California
18,ALAMEDA AND CONTRA COSTA COUNTIES AT VARIOUS LOCATIONS CROSSWALK SAFETY ENHANCEMENT,Training,California,Statewide,,Project,Project Statewide,Training for ___ in California
...,...,...,...,...,...,...,...,...
650,IN RIVERSIDE COUNTY NEAR MORENO VALLEY ON INTERSTATE 215 FROM 0.2 MILE NORTH OF ALESSANDRO BOULEVARD OVERCROSSING TO 0.2 MILE SOUTH OF EUCALYPTUS AVE,Training,California,Riverside County,,Project,Project in Riverside County,Training for ___ in California
653,"SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.",Training,California,Santa Clara County,,Project,Project in Santa Clara County,Training for ___ in California
656,"SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.",Training,California,Santa Clara County,,Project,Project in Santa Clara County,Training for ___ in California
661,IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE,Training,California,Los Angeles County,,Project,Project in Los Angeles County,Training for ___ in California


In [53]:
((utils.get_list_of_words((no_match_title>>filter(_.improvement_type_description=='Training')), "project_title")).value_counts()).iloc[0:40]

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


mile             64
county           50
route            35
near             31
north            25
road             25
south            25
los              23
san              22
west             22
02               20
angeles          19
overcrossing     18
avenue           16
boulevard        14
construct        13
santa            13
hma              13
locations        12
01               12
east             12
various          12
bridge           11
05               11
separation       10
undercrossing    10
miles            10
04                9
canyon            8
03                8
orange            7
city              7
street            6
counties          6
systems           6
install           6
replace           6
drainage          6
upgrade           6
safety            5
dtype: int64

In [54]:
# df>>filter(_.improvement_type_description=='Training')>>select(_.program_code_description, _.project_title, _.improvement_type_description,
#                                                               _.project_method, _.project_type, _.project_name_new)

In [55]:
## potential duplicates. different "descriptions 

df>>filter(_.project_title.str.contains("ERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58"))>>select(_.program_code_description, _.project_title, _.improvement_type_description,
                                                              _.project_method, _.project_type, _.project_name_new)

,program_code_description,project_title,improvement_type_description,project_method,project_type,project_name_new
398,NATIONAL HIGHWAY PERF IIJA,KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58,4R - No Added Capacity,,Project,Project in Kern County
399,NATIONAL HIGHWAY PERF IIJA,KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58,4R - No Added Capacity,,Project,Project in Kern County
400,NATIONAL HIGHWAY PERF IIJA,KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58,Bridge Replacement - No Added Capacity,,Project,Project in Kern County
401,NATIONAL HIGHWAY PERF IIJA,KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58,Bridge Replacement - No Added Capacity,,Project,Project in Kern County
402,NATIONAL HIGHWAY PERF IIJA,KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58,Training,,Project,Project in Kern County
403,NATIONAL HIGHWAY PERF IIJA,KERN COUNTY IN THE CITY OF BAKERSFIELD ON ROUTE 99 FROM 0.3 MILE SOUTH OF BELLE TERRACE OVERCROSSING TO 0.1 MILE NORTH OF ROUTE 58; ALSO ON ROUTE 58,Training,,Project,Project in Kern County


In [56]:
no_match_title >>filter(_.project_title.str.contains('BRIDGE'))>>select(_.project_title, _.improvement_type_description,
                                                              _.project_method, _.project_type, _.project_name_new, _.project_name_new2)>>filter(_.project_name_new2=="")

,project_title,improvement_type_description,project_method,project_type,project_name_new,project_name_new2


### Try with one element of project

- projects listed can have multiple entires bases on the phases they have. currently, if a project has multiple entries, the `improvement_type_description` has different description phrases, leading the title function to create a title that does not accuratley reflect the program
- what we want to do is apply the function to one entry and the use that accross the multiple projects

In [107]:
## How many projects have multiple funding componenets?
## a sample of those under the no_match flag with at least two entires in system

(no_match
 >>group_by(_.recipient_project_number, _.project_number, _.implementing_agency, _.project_title)
 >>summarize(n_descriptions = _.improvement_type_description.nunique())
 >>filter(_.n_descriptions>1)
 >>arrange(_.n_descriptions, _.n_descriptions)
)

,recipient_project_number,project_number,implementing_agency,project_title,n_descriptions
23,0113000123S,P020198,California,IN MENDOCINO COUNTY NEAR UKIAH FROM 0.3 MILE WEST OF RUSSIAN RIVER BRIDGE AND OVERHEAD TO 0.4 MILE EAST OF COUNTY ROAD 144 REPLACE TWO BRIDGES,2
27,0117000018S,P020199,California,"IN LAKE COUNTY NEAR CLEAR LAKE OAKS AT BERYL WAY THIS PROJECT PROPOSES TO INSTALL A COMBINATION OF TWO ALTERNATIVE IN-LINE TERMINAL SYSTEMS, A RAIL E",2
30,0213000002S,0059180,California,"IN SISKIYOU COUNTY, AT VARIOUS LOCATIONS FROM 0.2 MILE NORTH OF REAM AVENUE OVERCROSSING TO 0.2 MILE SOUTH OF HILT ROAD OVERCROSSING. ENHANCE TRAFFIC",2
31,0216000044S,0058644,California,IN TEHAMA COUNTY NEAR RED BLUFF FROM NINE MILE HILL OVERCROSSING TO 0.1 MILE NORTH OF BOWMAN ROAD OVERCROSSING. IMPROVE CLEAR RECOVERY ZONE,2
35,0300020624S,P020179,California,"NEAR LAKE WILDWOOD FROM 0.4 MILE EAST OF MCGANNERY LANE TO YUBA/NEVADA COUNTY LINE; ALSO IN NEVADA COUNTY, FROM YUBA/NEVADA COUNTY LINE TO MOONEY FLA",2
...,...,...,...,...,...
136,0615000046S,0055155,California,FRESNO COUNTY NEAR MENDOTA FROM 1.2 MILES NORTH OF HUDSON AVENUE OVERCROSSING TO 0.4 MILE SOUTH OF MANNING AVENUE OVERCROSSING. REPLACE TUMEY GULCH,4
158,0716000064S,0053074,California,"NEAR CASTAIC, FROM LAKE HUGHES ROAD UNDERCROSSING TO 1.9 MILES NORTH OF TEMPLIN HIGHWAY. REPLACE APPROACH SLABS, UPGRADE DRAINAGE SYSTEMS, UPGRADING",4
192,0900000030S,P395348,California,IN INYO COUNTY NEAR OLANCHA AND CARTAGO FROM 1.4 MILES SOUTH OF LOS ANGELES AQUEDUCT BRIDGE TO 0.2 MILE SOUTH OF ASH CREEK BRIDGE. CONSTRUCT 4 LANE E,4
197,1013000266S,S108057,California,"TUOLUMNE COUNTY IN AND NEAR LONG BARN FROM 0.1 MILE WEST OF LONG BARN ROAD TO 2.3 MILES EAST OF COW CREEK ROAD COLD PLANE, HMA, DRAINAGE REPLACEMENT,",4


* of the 133 projects wiht unique project IDs, 60 have more than 1 entry
* using all unique regardless of numbers 

In [136]:
df_copy = (df>>filter(_.project_type==('Project')))

In [109]:
len(df>>filter(_.project_type==('Project'))
 >>group_by(_.fmis_transaction_date, _.program_code, _.program_code_description, _.project_number,
                                                   _.recipient_project_number, _.project_title, _.county_code, _.congressional_district,
                                                   _.summary_recipient_defined_text_field_1_value, _.implementing_agency_locode,
                                                   _.implementing_agency, _.district, _.county_name, _.rtpa_name, _.mpo_name)
 >>summarize(n_descriptions = _.improvement_type_description.nunique())>>filter(_.n_descriptions>1)>>arrange(_.n_descriptions, _.n_descriptions))

60

In [110]:
len(df>>filter(_.project_type==('Project'))
 >>group_by(_.fmis_transaction_date, _.program_code, _.program_code_description, _.project_number,
                                                   _.recipient_project_number, _.project_title, _.county_code, _.congressional_district,
                                                   _.summary_recipient_defined_text_field_1_value, _.implementing_agency_locode,
                                                   _.implementing_agency, _.district, _.county_name, _.rtpa_name, _.mpo_name)
 >>summarize(n_descriptions = _.improvement_type_description.nunique()))

133

In [146]:
proj_unique = (df>>filter(_.project_type==('Project'))
#  >>group_by(_.fmis_transaction_date, _.program_code, _.program_code_description, _.project_number,
#                                                    _.recipient_project_number, _.project_title, _.county_code, _.congressional_district,
#                                                    _.summary_recipient_defined_text_field_1_value, _.implementing_agency_locode,
#                                                    _.implementing_agency, _.district, _.county_name, _.rtpa_name, _.mpo_name)
# >>arrange(_.project_number, _.obligations_amount)
              )

#### Use .max() to get the max value for obligation amount (the main portion of the funding)

In [147]:
#proj_unique.drop_duplicates('project_number', keep='last')
proj_unique = proj_unique.groupby(['fmis_transaction_date','program_code','project_number','recipient_project_number','project_title',
                    'implementing_agency', 'district', 'county_name'])['obligations_amount'].max().reset_index()

In [148]:
merge_cols = ['fmis_transaction_date', 'program_code','project_number','recipient_project_number','project_title',
                    'implementing_agency', 'county_name', 'district', 'obligations_amount']

#### Merge to get full details

In [149]:
proj_unique = (pd.merge(proj_unique, df_copy, how='left', on=merge_cols))

In [151]:
proj_unique.sample(4)

,fmis_transaction_date,program_code,project_number,recipient_project_number,project_title,implementing_agency,district,county_name,obligations_amount,program_code_description,county_code,congressional_district,improvement_type,improvement_type_description,summary_recipient_defined_text_field_1_value,implementing_agency_locode,rtpa_name,mpo_name,project_method,project_type,project_name_new
32,2022-04-26,Y001,NBIS522,0000001453L5,STATEWIDE - IN SERVICE BRIDGES OWNED BY LOCAL AGENCIES FEDERALLY MANDATED BRIDGE INSPECTION PROGRAM,California,3.00,Sacramento County,"14,164,800.00",NATIONAL HIGHWAY PERF IIJA,67.00,Cong Dist 3,49.00,Bridge Inspection and Bridge Related Training,S NON-MPO,NaN,NaN,NaN,,Project,Project in Sacramento County
204,2022-09-12,YS30,P204006,0617000297S,KERN COUNTY IN BAKERSFIELD AT 8TH STREET. INSTALLATION OF HIGH INTENSITY ACTIVATED CROSSWALK.,California,6.00,Kern County,"1,170,500.00",HIGHWAY SAFETY IMP PROG IIJA,29.00,Cong Dist 21,21.00,Safety,S KCOG,NaN,NaN,NaN,Install,Project,Install Project in Kern County
73,2022-05-16,Y001,P057069,0714000067S,"LA-057, IN LOS ANGELES COUNTY IN DIAMOND BAR AND CITY OF INDUSTRY FROM ORANGE COUNTY LINE TO ROUTE 57/60 SEPARATION THE PROJECT IS LOCATED IN LOS ANG",California,12.00,Orange County,"23,276.43",NATIONAL HIGHWAY PERF IIJA,59.00,Cong Dist 39,17.00,Construction Engineering,S SCAG,NaN,NaN,NaN,,Project,Project in Orange County
193,2022-09-09,Y240,5177047,0422000384L,"OYSTER POINT BLVD BETWEEN VETERANS BLVD AND ECCLES AVE, GATEWAY BLVD BETWEEN OYSTER POINT BLVD AND E GRAND AVE, AND EGRAND AVE BETWEEN GATEWAY BLVD A",South San Francisco,4.00,San Mateo County,"46,411.00",SURFAC TRNSP BLK GRTS-FLX IIJA,81.00,Cong Dist 14,17.00,Construction Engineering,L5177MTC,"5,177.00",Metropolitan Transportation Commission,Metropolitan Transportation Commission,,Project,Project in South San Francisco


#### run with update_no_matched function

In [160]:
proj_unique_new = update_no_matched(proj_unique, 'project_type', 'improvement_type_description', 'implementing_agency')

In [162]:
proj_unique_new.sample(5)

,fmis_transaction_date,program_code,project_number,recipient_project_number,project_title,implementing_agency,district,county_name,obligations_amount,program_code_description,county_code,congressional_district,improvement_type,improvement_type_description,summary_recipient_defined_text_field_1_value,implementing_agency_locode,rtpa_name,mpo_name,project_method,project_type,project_name_new,project_name_new2
51,2022-05-05,Y233,5946138,0614000078L,BRIDGE 46C0004 ON COUNTY ROAD D112 OVER NORTH BRANCH TULE RIVER REPLACE 2 LANE BRIDGE WITH 2 LANE BRIDGE (TC),Tulare County,6.00,Tulare County,"1,391,304.35",STBG IIJA OFF-SYSTEM BRIDGE,107.00,Cong Dist 22,11.00,Bridge Replacement - No Added Capacity,L5946TCAG,"5,946.00",Tulare County Association of Governments,Tulare County Association Of Governments,Replace,Project,Replace Project in Tulare County,Bridge Replacement in Tulare County
16,2022-03-07,Y301,5108202,0722000055L,"DOWNTOWN LONG BEACH (BOUNDED BY MAGNOLIA AVE TO THE WEST, OCEAN BLVD TO THE SOUTH, 7TH STREET TO THE NORTH, AND ALAMITOS AVE TO THE EAST). NON-INFRAS",Long Beach,7.00,Los Angeles County,"543,000.00",TRANS ALTERNATIVES >200K IIJA,37.00,Cong Dist 47,44.00,Other,L5108SCAG,"5,108.00",Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments,,Project,Project in Long Beach,
140,2022-07-25,YS30,5921080,0419000574L,"SIVERADO TRAIL: NORTH OF TRANCAS ST. AND SOUTH OF SR 29; EXCLUDING PORTIONS WITHIN JURISDICTION OF CITY OF ST. HELENA, CITY OF CALISTOGA, AND CALTRAN",Napa County,4.00,Napa County,"1,470,217.08",HIGHWAY SAFETY IMP PROG IIJA,55.00,Cong Dist 5,21.00,Safety,L5921MTC,"5,921.00",Metropolitan Transportation Commission,Metropolitan Transportation Commission,,Project,Project in Napa County,Safety Improvement Projects in Napa County
194,2022-09-09,YS30,5051005,0420000045L,"ON SILVERADO TRAIL, WITH THE PROJECT LIMITS BORDERING NAPA COUNTY, BEGINNING FROM APPROXIMATELY 175 FEET NORTH OF HOWELL MOUNTAIN ROAD AND ENDING AT",St Helena,4.00,Napa County,"66,500.00",HIGHWAY SAFETY IMP PROG IIJA,41.00,Cong Dist 2,21.00,Safety,L5051MTC,"5,051.00",Metropolitan Transportation Commission,Metropolitan Transportation Commission,,Project,Project in St Helena,Safety Improvement Projects in St Helena
63,2022-05-10,Y001,Q101396,0517000001S,SANTA BARBARA AND SAN LUIS OBISPO COUNTIES AT VARIOUS LOCATIONS FROM ROUTE 135 TO WEST TEFFT STREET CONSTRUCT CONTRASTING SURFACE TREATMENT AND GORE,California,5.00,San Luis Obispo County,"843,287.44",NATIONAL HIGHWAY PERF IIJA,79.00,Cong Dist 24,6.00,4R - Restoration & Rehabilitation,S SBCAG,NaN,NaN,NaN,Construct,Project,Construct Project in San Luis Obispo County,Road Restoration & Rehabilitation in California


In [164]:
len(proj_unique_new>>filter(_.project_name_new2==""))

24

In [165]:
(proj_unique_new>>filter(_.project_name_new2=="")>>select(_.project_number, _.recipient_project_number, _.project_title, _.improvement_type_description, _.implementing_agency, _.county_name,
                                                            _.project_method, _.project_type, _.project_name_new, _.project_name_new2))

,project_number,recipient_project_number,project_title,improvement_type_description,implementing_agency,county_name,project_method,project_type,project_name_new,project_name_new2
2,6480013,0417000451L,COUNTY WIDE- APPROXIMATELY 300 PUBLIC SCHOOL ALAMEDA COUNTY SAFE ROUTS TO SCHOOL PROGRAM OUTREACH AND EDUCATION. WORK PROGRAM IS FOCUSED ON SUPPORTIN,Safety and Education for Peds/Bicyclists,Alameda County Transportation Commission,Alameda County,,Project,Project in Alameda County Transportation Commission,
11,5917084,0314000112L,DOG BAR ROAD AT BEAR RIVER ~0.5 MILES SOUTH EAST OF MAGNOLIA ROAD AT COUNTY LINE. BR. # 17C0031 REPLACE 1 LANE BRIDGE WITH 2 LANE BRIDGE (TC),Right of Way,Nevada County,Nevada County,Replace,Project,Replace Project in Nevada County,
16,5108202,0722000055L,"DOWNTOWN LONG BEACH (BOUNDED BY MAGNOLIA AVE TO THE WEST, OCEAN BLVD TO THE SOUTH, 7TH STREET TO THE NORTH, AND ALAMITOS AVE TO THE EAST). NON-INFRAS",Other,Long Beach,Los Angeles County,,Project,Project in Long Beach,
20,6084277,0422000334L,REGIONWIDE - NINE COUNTY SAN FRANCISCO BAY AREA THE REGIONAL MAPPING DATA SERVICES SYSTEM WILL EXPOSE A SET OF PREDEFINED MAPPING-RELATED DATA APIS.(,Other,Metropolitan Transportation Commission,Multi-County,,Project,Project in Metropolitan Transportation Commission,
32,NBIS522,0000001453L5,STATEWIDE - IN SERVICE BRIDGES OWNED BY LOCAL AGENCIES FEDERALLY MANDATED BRIDGE INSPECTION PROGRAM,Bridge Inspection and Bridge Related Training,California,Sacramento County,,Project,Project in Sacramento County,
56,P001676,0716000335S9,"PROJECT ID 0716000335 (EA 07-4X970) LA-001-42.5/42.5 (PPNO-5059)ROUTE 001: IN MALIBU, SOUTH OF BIG ROCK DRIVE. SHORELINE EMBANKMENT RESTORATION",Right of Way,California,Los Angeles County,,Project,Project in Los Angeles County,
82,6084279,0422000413L,SF BAY AREA: REGIONWIDE TECHNICAL ASSISTANCE MOBILITY HUB PILOT PROGRAM(TC),Other,Metropolitan Transportation Commission,Multi-County,,Project,Project in Metropolitan Transportation Commission,
96,0052029,1216000044S,"ORANGE COUNTY, IN ANAHEIM FROM SANTA ANA STREET OVERCROSSING TO 0.5 MILE NORTH OF LA PALMA AVENUE OVERCROSSING PLANTING, IRRIGATION, ROCK BLANKET, MI",Other,California,Orange County,,Project,Project in Orange County,
103,4053001,0716000031S,IN LOS ANGELES COUNTY IN LOS ANGELES FROM 0.2 MILE SOUTH OF BURBANK BOULEVARD OVERCROSSING TO 0.3 MILE NORTH OF RINALDI STREET UNDERCROSSING PLANT VE,Other,California,Los Angeles County,,Project,Project in Los Angeles County,
104,P014105,0716000029S,"IN AND NEAR SANTA CLARITA, FROM CANYON PARK BOULEVARD TO SPRING CANYON ROAD UNDERCROSSING. MINOR CONCRETE BEYOND GORE, SLOPE PAVE BENEATH ABUTMENTS,",Other,California,Los Angeles County,Pave,Project,Pave Project in Los Angeles County,
